In [1]:
import pandas as pd
from util.readme_parser import ReadmeSectionParser
import regex
import string


In [2]:
# FIXME: Sections cleande contenti boş olan sectionlar siliniyor, ancak bu durumda gruplamada sorun oluşuyor, dikkat et.
repos = pd.read_csv("data/acl/scraping/papers_with_code_readme.csv")
sections = pd.read_csv("data/acl/sections/sections_clean.csv", index_col=0)


In [3]:
repos.shape, sections.shape


((9300, 8), (35463, 5))

In [4]:
repos.drop_duplicates(subset="code", inplace=True)
repos.shape


(7500, 8)

In [5]:
repos.sort_values(by="stars", ascending=False, inplace=True)


In [6]:
grouped_sections = []

for idx, repo in repos.iterrows():
    repo_sections = sections[sections["repo"] == repo.repo_dir]
    grouped = ReadmeSectionParser.group_by_parent_header(repo_sections)
    grouped_sections += grouped
    if len(grouped_sections) > 1000:
        break

grouped_sections = pd.DataFrame(grouped_sections)


In [7]:
selected_repos = repos[repos.repo_dir.isin(grouped_sections.repo.unique())]
selected_repos.shape


(212, 8)

In [8]:
grouped_sections = grouped_sections.assign(stars=grouped_sections["repo"].map(
    selected_repos.set_index("repo_dir")["stars"]).astype(int))
grouped_sections.shape


(1004, 4)

In [9]:
grouped_sections.rename(columns={"content": "text"}, inplace=True)
grouped_sections = grouped_sections.assign(label="")


In [10]:
# Remove chinese or other non-ascii characters
grouped_sections.text = grouped_sections.text.apply(
    lambda x: x.encode("ascii", errors="ignore").decode())


In [12]:
grouped_sections[["repo", "header", "text", "label", "stars"]].to_json(
    "data/acl/sections/manuel_labelling/grouped_1000_orderby-stars.json", orient="records")
